In [ ]:
# Solving the model in Aiyagari (1994) 
# 2024.12.03
#######################################
# This Python script incorporates insights from the ComputaitonLab 
# (https://github.com/hanbaeklee/ComputationLab/tree/main) by Hanbaek Lee (hanbaeklee1@gmail.com). All errors are my own.
#######################################
# Author: Iman Taghaddosinejad
#######################################
# Overview of Algorithm: 
# 1. Guess Aggregate Allocation(s) (K) to pin prices 
# 2. Solve individual oprimization problem (VFI + Howard Improvement) to derive policy rules 
# 3. Using a non-stochastic iterative (histogram) method (Young, 2010) derive a stationary distribution over states 
# 4. Update prices and aggregate allocation  
# 5. Repeat steps 2-4 until aggregate allocation (and so prices) converge
#######################################

In [204]:
## Load Packages ## 

import pandas as pd 
import quantecon as qe
import numpy as np 
from scipy.optimize import minimize_scalar
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt 

In [205]:
## Set Parameters ## 

pA = 1 # TFP 
pAlpha = 0.36 # income share of capital (Cobb-Douglas)
pBeta = 0.96 # annual frequency 
pDelta = 0.08 # depreciation rate 
pRho = 0.9 # logAR1 persistence coefficient 
pSigmaz = 0.0872 
Nz = 7 # number of states for z 
Na1 = 50 # number of asset/wealth states - sparse grid  
Na2 = 100 # number of asset/wealth states - fine grid 
minGrida = 0 # lower bound on asset grid (savings constraint) 
maxGrida = 150 # upper bound for wealth - large enough to not constrain optimal policy

In [206]:
## Helper Functions ##

# objective function - HH maximization problem for given state (a,z) w/ a single control (a') 
def fnObj(x, grida, expval, beta, M, N):    
    aprime = x # set optimal savings 
    
    aLow = np.searchsorted(grida, aprime, side="right") - 1 # closest point (below) to aprime in asset-grid
    aLow = max(0, min(aLow, N - 2)) # snap to index zero (N-1) if aprime falls below (above) the asset-grid 
    aHigh = aLow + 1

    wtLow = (grida[aHigh] - aprime) / (grida[aHigh] - grida[aLow])
    wtLow = np.clip(wtLow, 0, 1) # snap to boundary zero (one) whenever aprime is above (below) the asset-grid 

    expVal = wtLow*expval[aLow] + (1 - wtLow)*expval[aHigh]
    c = max(1e-8, M-aprime)

    Val = np.log(c) + beta * expVal # assumes log-utility 
    return -Val 

# optimizer - a minimization solver applied over HH objective function which returns optimal a'  
def fnOptaprime(vGrida1, expVF, pBeta, cash, Na1, minwealth):
    result = minimize_scalar(
        lambda x: fnObj(x, grida=vGrida1, expval=expVF, beta=pBeta, M=cash, N=Na1),
        bounds=(minwealth, cash),
        method='bounded')
    return result.x

# function - compute stationary dist. of a First-Order Markov process
def mc_invariant_dist(Tmat, eigvalmethod=False):
    if eigvalmethod == True:
        eigvl, eigvc = np.linalg.eig(Tmat)
        index = np.argmin(np.abs(eigvl - 1))
        eigvc = eigvc[:, index] / np.sum(eigvc[:, index]) 
        #print("Stationary Distribution:", np.round(eigvc, 4))
        return eigvc
    else:
        x0 = np.zeros(len(Tmat))
        x0[0] = 1
        tol = 1e-6
        err = 1 
        while err > tol:
            x = x0 
            y = Tmat @ x
            err = np.linalg.norm(y-x)
            x0 = y
        #print("Stationary Distribution", np.round(x0, 4))
        return x0

In [207]:
## Define Grids ##

# wealth grid - coarsed (dense near lower limit) grid both sparse and finer 
x1 = np.linspace(0, 0.5, num=Na1)
x2 = np.linspace(0, 0.5, num=Na2)
y1 = x1**7 / np.max(x1**7)
y2 = x2**7 / np.max(x2**7)
vGrida1, vGrida2 = minGrida + (maxGrida - minGrida)*y1, minGrida + (maxGrida - minGrida)*y2

# productivity grid - discretize log-AR1 
mc = qe.markov.approximation.tauchen(n=Nz, rho=pRho, sigma=((pSigmaz**2) * (1-pRho**2)), mu=0.0, n_std=3)
mTz, vGridz = mc.P, np.exp(mc.state_values) # transition matrix and state(s) 
vPi_z = mc_invariant_dist(Tmat=mTz.T, eigvalmethod=True) # stationary distribution 

In [208]:
## Initialize GE Loop ##

initK = 6 # initial guess for aggregate capital 
initmVF = np.tile(0.01*vGrida1, (Nz, 1)).T # NaxNz placeholder for initial value function guess (appx zero)
initmDist = np.ones((Na2, Nz)) / (Na2 * Nz) # statrionary distribution guess (uniform over all (a,z) states )

mVFnew = np.zeros_like(initmVF) # final Value Function 
mPolc = np.zeros_like(initmVF) # consumption policy function 
mPolaprime1 = np.zeros((Na1, Nz)) # aprime (a') policy function define over sparse grid 
mPolaprime2 = np.zeros((Na2, Nz)) # aprime (a') policy function defined over finer grid

# set loop parameters 
#err_margin = 10 # set initial error 
errtol_outer = 1e-6 # outer GE loop error tolerance
errtol_inner = 1e-6 # inner GE loop error tolerance 
errDistTol = 1e-8 # stationary distribution error tolerance

errVF = 10 
errDist = 10

itercount1 = 1 # iteration counter 
wt_old = 0.9900 # weight assigned to initial guess in updating outer loop variables/prices  
maxiter_outer = 3000 # outer GE loop max iteration limit 
maxiter_inner = 2000 # inner GE loop max iteration limit 
converged_outer = False 
converged_inner = False 

In [ ]:
mPolaprime_HL = np.loadtxt(r'/Users/iman/Documents/Cam_PhD_Econ/PhD_Workshops/ComputationLab/aiyagari1994/mPolaprime.csv', delimiter=',')
mPolc_HL = np.loadtxt(r'/Users/iman/Documents/Cam_PhD_Econ/PhD_Workshops/ComputationLab/aiyagari1994/mPolc.csv', delimiter=',')
mValue_HL = np.loadtxt(r'/Users/iman/Documents/Cam_PhD_Econ/PhD_Workshops/ComputationLab/aiyagari1994/mValue.csv', delimiter=',')

In [209]:
## Solve RE-SRCE ##

#==================================================
# OUTER LOOP 
#==================================================
  
# Define Objects Needed Before Iteration Loop(s) 
aggK = initK # initial guess for aggregate capital
aggL = vGridz.T @ vPi_z # inelastic labour supply 
currDist = np.copy(initmDist) # initialize distribution to start the iteration
mVF = np.copy(initmVF) # initialize value function to start the iteration

# GE loop
while converged_outer!=True and itercount1<=maxiter_inner:

    # set prices - given aggregates
    aggKL_ratio = aggK/aggL  
    r = pAlpha * pA * (aggK/aggL)**(pAlpha-1) - pDelta 
    w = (1-pAlpha) * pA * (aggK/aggL)**(pAlpha)
    mmu = r+pDelta

    #==================================================
    # INNER LOOP - w/ Howard Improvement
    #================================================== 

    itercount2 = 1
    errVF = 10
    converged_inner = False  # Flag for convergence

    # VFI loop
    while not converged_inner:
        #print(f"Iteration {itercount2} in progress...")

        for iz, z in enumerate(vGridz):

            # Compute expected value of Value Function in Bellman Eq
            expVF = mVF @ mTz.T
            expVF = expVF[:, iz]  # Extract column for the current z state

            # reset min. value of wealth - use monotonicity in policy rule of a' to speed up optimal search
            minwealth = minGrida
            
            for ia, a in enumerate(vGrida1):

                #---------- SOLVING FOR OPTIMAL POLICY FUNCTION/RULE ----------#    
                if itercount2 <= 20 or itercount2 % 20 == 0:  
                    
                    cash = w*z + (1+r)*a  # Cash-in-hand given state (a, z) - upper limit for a'
                    aprime = fnOptaprime(vGrida1, expVF, pBeta, cash, Na1, minwealth)
                    
                    # savings bounded below by borrowing constraint
                    aprime = max(aprime, minGrida)   
                    
                    # Interpolate aprime on the asset-grid
                    aLow = np.searchsorted(vGrida1, aprime, side="right") - 1
                    aLow = max(0, min(aLow, Na1 - 2))
                    aHigh = aLow + 1
                    wtLow = (vGrida1[aHigh] - aprime) / (vGrida1[aHigh] - vGrida1[aLow])
                    wtLow = np.clip(wtLow, 0, 1)

                    # Using interpolated aprime, interpolate expected VF
                    expVal = wtLow*expVF[aLow] + (1-wtLow)*expVF[aHigh]
                    c = cash - aprime

                    # Update VF and PFs
                    minwealth = aprime 
                    mVFnew[ia, iz] = np.log(c) + pBeta*expVal
                    mPolc[ia, iz] = c
                    mPolaprime1[ia, iz] = aprime
    
                #---------- UPDATING VF USING ACCELLERATOR - HOWARD IMPROVEMENT ----------#
                else:    
                    
                    aprime = mPolaprime1[ia, iz]
                    c = mPolc[ia, iz]

                    aLow = np.searchsorted(vGrida1, aprime, side="right") - 1
                    aLow = max(0, min(aLow, Na1 - 2))
                    aHigh = aLow + 1
                    wtLow = (vGrida1[aHigh] - aprime) / (vGrida1[aHigh] - vGrida1[aLow])
                    wtLow = np.clip(wtLow, 0, 1)

                    expVal = wtLow*expVF[aLow] + (1-wtLow)*expVF[aHigh]
                    mVFnew[ia, iz] = np.log(c) + pBeta*expVal # update VF using Howard Improvement
    
        # Compute distance between old and new VF
        errVF = np.max(np.abs(mVFnew - mVF))

        if errVF <= errtol_inner:
            converged_inner = True
        else:
            mVF = np.copy(mVFnew) # Update value function to the newly computed one
        
        itercount2 += 1  # Increment iteration counter
    #print(f"VFI Converged: {converged_inner}. After {itercount2} Iterations! \n")

    # Interpolate Savings (a') Policy Function on Finer Asset Grid
    for iz in range(Nz):
        FnInterp = interp1d(x=vGrida1, y=mPolaprime1[:, iz], kind='linear', fill_value='extrapolate')
        mPolaprime2[:, iz] = FnInterp(vGrida2)

    # Aggregation - Compute Stationary Distribution (Non-Stochastic Iterative Method)
    errDist = 10
    while errDist >= 1e-8:
        nextDist = np.zeros_like(currDist) # reset next distribution to fill with updated mass
    
        for iz in range(Nz):
            for ia in range(Na2):
                
                anext = mPolaprime2[ia, iz]
                LB = np.searchsorted(vGrida2, anext, side="right") - 1  
                LB = max(0, min(LB, Na2 - 2))
                UB = LB + 1 
                wtLB = (vGrida2[UB] - anext) / (vGrida2[UB] - vGrida2[LB])
                wtLB = np.clip(wtLB, 0, 1)
                wtUB = 1 - wtLB

                mass = currDist[ia, iz]
 
                # Incrementally Change Next Period Distribution     
                for iznext in range(Nz):
                    nextDist[LB, iznext] = nextDist[LB, iznext] +       mass*mTz[iz, iznext]*wtLB
                    nextDist[UB, iznext] = nextDist[UB, iznext] +       mass*mTz[iz, iznext]*wtUB
                    
        # Compute Error 
        errDist = np.max(np.abs(nextDist - currDist))

        # Update Distribution
        currDist = nextDist.copy()

    # Update Aggregate(s)/Price(s)
    marginalDista = np.sum(nextDist, axis=1) # sum across columns for marginal dist over asset-grid 
    aggKnew = vGrida2.T @ marginalDista # new aggregate capital

    # Check for Convergence (Outer Loop)
    errK = np.abs(aggKnew - aggK)
    if errK <= errtol_outer:
        converged_outer = True 
    else:
        aggK = wt_old*aggK + (1 - wt_old)*aggKnew # smoothly update agg captial
    
    # Print Loop Progress
    if itercount1 % 1 == 0 or itercount1 == 1:
        print(f"\n")
        print(f"Market Clearing Report (Outer Loop Iteration {itercount1}):")
        print(f"AggK: {np.round(aggK, 5)}, error: {np.round(errK, 8)}")
        print(f"Prices: r = {np.round(r, 4)}, w = {np.round(w, 4)}")
        print("-" * 80)
    itercount1 += 1






Market Clearing Report (Outer Loop Iteration 1):
AggK: 5.94, error: 5.99997586
Prices: r = 0.0344, w = 1.2199
--------------------------------------------------------------------------------


Market Clearing Report (Outer Loop Iteration 2):
AggK: 5.8806, error: 5.93997639
Prices: r = 0.0351, w = 1.2155
--------------------------------------------------------------------------------


Market Clearing Report (Outer Loop Iteration 3):
AggK: 5.82179, error: 5.88057673
Prices: r = 0.0358, w = 1.2111
--------------------------------------------------------------------------------


Market Clearing Report (Outer Loop Iteration 4):
AggK: 5.76358, error: 5.82177095
Prices: r = 0.0366, w = 1.2067
--------------------------------------------------------------------------------


Market Clearing Report (Outer Loop Iteration 5):
AggK: 5.70594, error: 5.76355313
Prices: r = 0.0373, w = 1.2023
--------------------------------------------------------------------------------


Market Clearing Report

KeyboardInterrupt: 